In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from env import api_key

pd.set_option("display.max_rows", None, "display.max_columns", None)

pd.reset_option("display.max_rows", "display.max_columns")

import re

In [2]:
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/battlebud?api_key={api_key}")

In [3]:
battlebud = response.json()

In [4]:
battlebud

{'status': {'message': 'Forbidden', 'status_code': 403}}

In [5]:
url = 'https://americas.api.riotgames.com/'

query = f'lol/match/v5/matches/by-puuid/8w5OLGrYy-YhUvX08ZD6a4sBvBvZuYTXU6UnfYhC1MBNo8rlV4OEODS9bNrqMBVuWirDBYOeMhWW0Q/ids?start=0&count=20&api_key={api_key}'

In [6]:
response = requests.get(url+query)

In [7]:
match_ids = response.json()

In [8]:
len(match_ids)

1

In [9]:
match_ids

{'status': {'message': 'Forbidden', 'status_code': 403}}

In [10]:
url = 'https://na.op.gg/ranking/ladder/'
response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
soup = BeautifulSoup(response.text)
table = soup.select('.ranking-table')
table = table[0]
table = soup.select('.ranking-table__cell')

In [11]:
table

[<td class="ranking-table__cell ranking-table__cell--rank">6</td>,
 <td class="ranking-table__cell ranking-table__cell--summoner"><a href="//na.op.gg/summoner/userName=Johnsun123"><img onerror="this.src='//opgg-static.akamaized.net/images/profile_icons/profileIcondefault.jpg';" src="//opgg-static.akamaized.net/images/profile_icons/profileIcon3785.jpg?image=c_scale,q_auto,c_scale,w_38&amp;v=1518361200"/><span>Johnsun123</span></a></td>,
 <td class="ranking-table__cell ranking-table__cell--tier">
 									Challenger
 								</td>,
 <td class="ranking-table__cell ranking-table__cell--lp">
 									1,517 LP
 								</td>,
 <td class="ranking-table__cell ranking-table__cell--level">
 								63
 							</td>,
 <td class="ranking-table__cell ranking-table__cell--winratio"><div class="winratio"><div class="winratio-graph"><div class="winratio-graph__fill winratio-graph__fill--left" style="width: 61%"></div><div class="winratio-graph__text winratio-graph__text--left">206</div><div class="w

In [12]:
def get_username(table):
    username_list = []
    for i in range (1,571,6):
        name = str(table[i])
        username = str(re.findall(r'userName=.+?"', name))
        username_list.append(username)
        df = pd.DataFrame(username_list)
        df = df.rename(columns={0:"pro"})
        df['pro'] = df.pro.str.replace("\['userName=","")
        df['pro'] = df.pro.str.replace("+",' ')
        df['pro'] = df.pro.str.replace("\"']",'')
        df = df[~df.pro.str.contains('%')]
        df = df.reset_index()
    return df

In [42]:
def op_gg_webscrape(pages):
    player_names = []
    for i in range(pages + 1):
        url = (f'https://na.op.gg/ranking/ladder/page={i}')
        response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
        soup = BeautifulSoup(response.text)
        table = soup.select('.ranking-table')
        table = table[0]
        table = soup.select('.ranking-table__cell')
        users_df = get_username(table)
        for pro in users_df.pro:
                player_names.append(str(pro))
    return player_names

In [43]:
player_list = op_gg_webscrape(10)

<ipython-input-12-c953d951acde>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\['userName=","")
<ipython-input-12-c953d951acde>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['pro'] = df.pro.str.replace("+",' ')
<ipython-input-12-c953d951acde>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\"']",'')


In [44]:
player_list = list(set(player_list))

In [47]:
len(player_list)

890

In [49]:
player_df = pd.DataFrame({'players':player_list})

In [51]:
player_df.to_csv('more_players.csv')

In [28]:
def op_gg_webscrape(pages):
    player_names = []
    url = 'https://na.op.gg/ranking/ladder/'
    response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
    soup = BeautifulSoup(response.text)
    table = soup.select('.ranking-table')
    table = table[0]
    table = soup.select('.ranking-table__cell')
    users_df = get_username(table)
    for pro in users_df.pro:
        player_names.append(pro)
        for i in range(pages + 1):
            url = (f'https://na.op.gg/ranking/ladder/page={i}')
            response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
            soup = BeautifulSoup(response.text)
            table = soup.select('.ranking-table')
            table = table[0]
            table = soup.select('.ranking-table__cell')
            users_df = get_username(table)
            for pro in users_df.pro:
                player_names.append(pro)
    return player_names

In [ ]:
player_list = op_gg_webscrape(4)

<ipython-input-24-c953d951acde>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\['userName=","")
<ipython-input-24-c953d951acde>:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['pro'] = df.pro.str.replace("+",' ')
<ipython-input-24-c953d951acde>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\"']",'')


In [ ]:
player_list

In [158]:
player_names = []
url = 'https://na.op.gg/ranking/ladder/'
response = requests.get(url, headers = {'user-agent': 'https://github.com/JaredVahle'})
soup = BeautifulSoup(response.text)
table = soup.select('.ranking-table')
table = table[0]
names = soup.select('.ranking-table__cell')
for i in range (1,571,6):
    #summoner_names = re.match(r'userName=.*',str(names[i]))
    name = str(names[i])
    username = str(re.findall(r'userName=.+?"', name))
    player_names.append(username)

In [159]:
Tçº

SyntaxError: invalid syntax (<ipython-input-159-1847a16345a5>, line 1)

In [221]:
df = pd.DataFrame(player_names)

In [222]:
df = df.rename(columns={0:"pro"})

In [223]:
df

,pro
0,"['userName=Johnsun123""']"
1,"['userName=Qitong""']"
2,"['userName=Davemon""']"
3,"['userName=Sheiden2""']"
4,"['userName=Trevor26""']"
...,...
90,"['userName=Z+Score""']"
91,"['userName=wx+NAtongtiandai""']"
92,"['userName=Arc+en+ciel""']"
93,"['userName=Kittisaur""']"


In [224]:
df['pro'] = df.pro.str.replace("\['userName=","")

<ipython-input-224-480bfac5a110>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\['userName=","")


In [225]:
df['pro'] = df.pro.str.replace("+",' ')

<ipython-input-225-6bb7840fc7d9>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['pro'] = df.pro.str.replace("+",' ')


In [226]:
df['pro'] = df.pro.str.replace("\"']",'')

<ipython-input-226-504d2adf1d63>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['pro'] = df.pro.str.replace("\"']",'')


In [227]:
df.shape

(95, 1)

In [206]:
df['pro'] = df.where(df.pro.str.contains('%') == False)

In [231]:
df = df[~df.pro.str.contains('%')]

In [232]:
df

,pro
0,Johnsun123
1,Qitong
2,Davemon
3,Sheiden2
4,Trevor26
...,...
90,Z Score
91,wx NAtongtiandai
92,Arc en ciel
93,Kittisaur


In [228]:
df.sample(10)

,pro
33,Vakin
11,LeenK LCS
86,SkyTec
36,TempestLoveYuely
28,Ne%C3%B8%C3%B8
63,Karasmai v9
42,Feather Daddy
23,Sophist Sage
73,dfgtredfg4
8,Dhokla


In [64]:
len_list = []
for i in range (1,572,6):
    len_list.append(i)

In [79]:
name = str(names[1])

In [99]:
name

'<td class="ranking-table__cell ranking-table__cell--summoner"><a href="//na.op.gg/summoner/userName=Johnsun123"><img onerror="this.src=\'//opgg-static.akamaized.net/images/profile_icons/profileIcondefault.jpg\';" src="//opgg-static.akamaized.net/images/profile_icons/profileIcon3785.jpg?image=c_scale,q_auto,c_scale,w_38&amp;v=1518361200"/><span>Johnsun123</span></a></td>'

In [118]:
thing = name[100:116]

In [119]:
thing

'Johnsun123"><img'

In [134]:
re.match(r'^("userName=")(.+?")$',name)

In [146]:
username = str(re.findall(r'userName=.+?"', name))

In [147]:
username

'[\'userName=Johnsun123"\']'

In [148]:
username = username[11:-4]

In [149]:
username

'Johnsun12'

In [ ]:
username[-]

In [130]:
username

In [29]:
table[0]

KeyError: 0

In [10]:
name

[]

In [ ]:
def get_puuid(name_list):
    response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/battlebud?api_key={api_key}")
    summoner_info = response.json()

In [ ]:
response = requests.get(f"https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-name/battlebud?api_key={api_key}")

In [ ]:
summoner_info = response.json()

In [ ]:
battlebud